In [1]:
import torch
import clip
from PIL import Image
import argparse
import os
from torchvision import datasets, transforms
import numpy as np
import sys
from scipy import linalg
from tqdm import tqdm
from torch.distributions.normal import Normal
from scipy.stats import multivariate_normal
import shutil
import matplotlib.pyplot as plt

/home/toluwani/anaconda3/envs/hawau/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import glob
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.neighbors import LocalOutlierFactor

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Get all categories

In [3]:
categories = os.listdir('/home/toluwani/Documents/Hawau/Anomaly/data/')
categories.remove('license.txt')
categories.remove('readme.txt')

In [5]:
categories

['capsule',
 'leather',
 'tile',
 'zipper',
 'toothbrush',
 'screw',
 'hazelnut',
 'wood',
 'carpet',
 'metal_nut',
 'transistor',
 'bottle',
 'grid',
 'pill',
 'cable']

In [4]:
data = '/home/toluwani/Documents/Hawau/Anomaly/data/'

In [9]:
model, preprocess = clip.load("ViT-B/32")
model.to(device).eval()
input_resolution = model.visual.input_resolution

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.




### Load all data classes

In [18]:
df = {}
for cat in categories:
    train_path = f'{data}/{cat}/train/good/*.png'
    test_path = f'{data}/{cat}/test/'
    test_folders = os.listdir(data + cat + '/test')

    train_data = []
    for path in glob.iglob(os.path.abspath(train_path)):
        img = Image.open(path)
        train_data.append(preprocess(img))

    test_data= {'paths':[],'labels':[]}
    for folder in test_folders:
        for path in glob.iglob(os.path.abspath(test_path + folder + '/*.png')):
            img = Image.open(path)
            test_data['paths'].append(preprocess(img))
            if folder == 'good':
                test_data['labels'].append(1)    
            else:
                test_data['labels'].append(-1)


    df[cat] = {'train' : train_data , 'test' : test_data}
   
    

### Encode all class data

In [19]:
for cat in categories:
    image_input = torch.tensor(np.stack(df[cat]['train'])).cuda()
    image_test = torch.tensor(np.stack(df[cat]['test']['paths'])).cuda()

    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        test_features = model.encode_image(image_test).float()
    
    df[cat]['train'] = image_features.cpu().numpy()
    df[cat]['test']['paths'] = test_features.cpu().numpy()
     

### Train model and test accuracy

In [28]:
results = {}
for cat in categories:
    clf = IsolationForest(random_state=0).fit(df[cat]['train'])
    accclf = accuracy_score(df[cat]['test']['labels'],clf.predict(df[cat]['test']['paths']))


    lof = LocalOutlierFactor(n_neighbors=2, novelty=True, metric='manhattan')
    lof.fit(df[cat]['train'])
    acclof =accuracy_score(df[cat]['test']['labels'],lof.predict(df[cat]['test']['paths']))


    results[cat] = {'model' :['Isolation Forest','Local Outlier Factor'], 'accuracy' : [accclf,acclof]}


    del clf,lof



### Record 

In [29]:
import pandas as pd
pd.DataFrame.from_dict(results, orient='index')

,model,accuracy
capsule,"[Isolation Forest, Local Outlier Factor]","[0.44696969696969696, 0.4318181818181818]"
leather,"[Isolation Forest, Local Outlier Factor]","[0.9354838709677419, 0.9596774193548387]"
tile,"[Isolation Forest, Local Outlier Factor]","[0.9316239316239316, 0.7863247863247863]"
zipper,"[Isolation Forest, Local Outlier Factor]","[0.6887417218543046, 0.45695364238410596]"
toothbrush,"[Isolation Forest, Local Outlier Factor]","[0.6904761904761905, 0.5476190476190477]"
screw,"[Isolation Forest, Local Outlier Factor]","[0.2875, 0.275]"
hazelnut,"[Isolation Forest, Local Outlier Factor]","[0.5909090909090909, 0.5818181818181818]"
wood,"[Isolation Forest, Local Outlier Factor]","[0.9493670886075949, 0.9240506329113924]"
carpet,"[Isolation Forest, Local Outlier Factor]","[0.8632478632478633, 0.8717948717948718]"
metal_nut,"[Isolation Forest, Local Outlier Factor]","[0.5304347826086957, 0.43478260869565216]"
